<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/LANL-logo.png" alt="LANL Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/prowess.png" alt="Prowess Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/wildfire.png" alt="WildFire Logo" width="100"/>
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">Fire-Ready Forests Data Challenge</h1>

# Predicting Tree Metrics with ALS

In Sprint 2, we created a sample tree list derived from ALS (Airborne Laser Scanning). As explored in the associated module, the ALS-derived treelist provides valuable information about tree locations and heights. However, it lacks key attributes such as diameter, species, and crown base height—details essential not only for ecological and forestry applications but also for fire modeling. These metrics play a crucial role in predicting fire behavior: tree species influence flammability, diameter affects fuel availability, and crown base height determines vertical fire spread. In contrast, datasets like the FIA offer a more complete set of tree metrics but are based on sampled data and do not capture the full distribution of trees across the landscape.

In this notebook, we will explore how to use an ALS-derived treelist to predict missing tree attributes. Using a simple random forest model, we will estimate tree diameter, species, and crown base height.

For our training data, we will use the FastFuels treelist.

In [ ]:
# import packages

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Load the detected trees from the ALS data and the fast fuels treelist

ALS_treetops = pd.read_csv('./ALS_treetops.csv')
FF_treelist = pd.read_csv('FF_treelist_all.csv') # This assumes you did not change the path in the previous notebook
fia_ref_species_table = pd.read_csv('./REF_SPECIES.csv')

In [ ]:
# Quick look at the ALS treelist
ALS_treetops.head()

In [ ]:
# Quick look at the FastFuels treelist
FF_treelist.head()

In [ ]:
# let's look specifically at Independence Lake
# site_shortname = ['ind', 'SHA', 'SDR', 'win', 'puc', 'tcu']
site_name = 'ind'

In [ ]:
ALS_treetops_filter = ALS_treetops[ALS_treetops.site_name == site_name]
FF_treelist_filter = FF_treelist[FF_treelist.site_name == site_name]

# rename the height column for the ALS file
ALS_treetops_filter = ALS_treetops_filter.rename(columns = {'height_m': 'HT'})


#now turn into geodataframe

ALS_treetops_gdf = gpd.GeoDataFrame(ALS_treetops_filter, 
                                    geometry = gpd.points_from_xy(ALS_treetops_filter.X_4326, ALS_treetops_filter.Y_4326), 
                                    crs = 4326)

treelist = FF_treelist_filter

#first we need to rename one of our variables


We will train a different model for each variable that we are interested in predicting. 
This means that we will have a model to predict diameter from height, a second model to predict species from height, and a third model to predict crown base height from total height.

## Diameter from height

Our first random forest model will take in height (HT) as input and produce diameter (DIA) as output.

In [ ]:
# Split into training and test sets
independent_variables = ["HT"]
dependent_variable = "DIA"
include_variables = independent_variables + [dependent_variable]
trees_train, trees_test = train_test_split(treelist[include_variables].dropna(), test_size=0.2)
print(f"Training set size: {len(trees_train)}")
print(f"Test set size: {len(trees_test)}")

In [ ]:
# Train the model
model_ht = RandomForestRegressor()
model_ht.fit(trees_train[independent_variables], trees_train[dependent_variable])

# Predict the model
predicted_dbh_test = model_ht.predict(trees_test[independent_variables])
predicted_dbh = model_ht.predict(ALS_treetops_filter[independent_variables])

# Compute R^2 and RMSE using the test set
print(f"Model R^2: {model_ht.score(trees_test[independent_variables], trees_test[dependent_variable]):.2f}")
print(f"Model RMSE: {((model_ht.predict(trees_test[independent_variables]) - trees_test[dependent_variable])**2).mean()**0.5:.2f} inches")


# Compare the predicted diameter to the actual diameter.
fig, ax = plt.subplots()
trees_test["predicted_diameter"] = predicted_dbh_test
trees_test.plot.scatter(x="DIA", y="predicted_diameter", ax=ax)
upper_dia_limit = max(trees_test["DIA"].max(), trees_test["predicted_diameter"].max()) + 1
ax.plot([0, upper_dia_limit], [0, upper_dia_limit], color='red')
ax.set_xlabel("Actual Diameter (cm)")
ax.set_ylabel("Predicted Diameter (cm)")
ax.set_title("Validation Data Diameter Prediction")
plt.show()

That's pretty good for a simple model! Accuracy decreases as diameter increases, but up to around 30 inches it is quite accurate. It may be worth investigating approaches for outlier detection and removal. For example, maybe we can improve model performance by only considering diameter estimates in a certain range.

Next, let's use our new model to predict the diameters using the heights we observed from the ALS LIDAR data.

In [ ]:
# User our new model to predict the diameter of the trees detected in the ALS data

ALS_treetops_filter["predicted_diameter"] = predicted_dbh

In [ ]:
# Plot the predicted diameter
fig, ax = plt.subplots()
ALS_treetops_filter.plot.scatter(x="HT", y="predicted_diameter", ax=ax)
ax.set_xlabel("Height (m)")
ax.set_ylabel("Predicted Diameter (cm)")
ax.set_title("Independence Lake Tree Diameter Prediction")
plt.show()

### SPCD (Species code) from height

Species code, or SPCD, is a numeric identifier for tree species across the United States. Tree species is an enormously important characteristic for making predictions about tree biomass, carbon content, size, and more. Unfortunately, we don't learn tree species from the ALS acquistion data so we want to try and predict it using a model trained on FIA data. In this example, we train a simple random forest classifer to predict trees species based just on the height of the tree. Let's see how well this simple model performs.

In [ ]:
# Create a dictionary mapping SPCD to COMMON_NAME
spcd_to_common_name = dict(zip(fia_ref_species_table['SPCD'], fia_ref_species_table['COMMON_NAME']))

In [ ]:
# Split into training and test sets
independent_variables = ["HT"]
dependent_variable = "SPCD"
include_variables = independent_variables + [dependent_variable]
trees_train, trees_test = train_test_split(treelist[include_variables].dropna(), test_size=0.2)
print(f"Training set size: {len(trees_train)}")
print(f"Test set size: {len(trees_test)}")

# Train the model
model_spcd = RandomForestClassifier()
model_spcd.fit(trees_train[independent_variables], trees_train[dependent_variable])

# Predict the model
predicted_spcd_test = model_spcd.predict(trees_test[independent_variables])
predicted_spcd = model_spcd.predict(ALS_treetops_filter[independent_variables])


# Evaluate classification accuracy
unique_species = sorted(trees_test[dependent_variable].unique())
species_names = [spcd_to_common_name.get(spcd, f"Unknown ({spcd})") for spcd in unique_species]
report = classification_report(trees_test[dependent_variable], predicted_spcd_test, zero_division=0, target_names=species_names)
print(report)

In [ ]:
cm = confusion_matrix(trees_test[dependent_variable], predicted_spcd_test)
display = ConfusionMatrixDisplay(cm, display_labels=species_names)
display.plot(cmap='Blues', values_format='d')
plt.xticks(rotation=90, ha='right')
plt.show()

It looks like the confusion matrix has identified some common areas of confusion for our model. 

The model frequently predicts lodgepole pine for different conifer trees including Ponderosa Pine, California red fir, and white fir, among other fir and pine species.

What can we do to our model to improve predictions? Are there additional variables that you can think of that would help with the prediction?


### Crown Base Height (CBH) from Height

Crown base height (CBH), sometimes called live crown base height, is a measurment of how far above the ground the crown of the tree is. You can think of this as if you stood under a tree and stretched and stretched until your fingertips touch leaves or needles. How far you have to stretch (including your Go-Go-Gadget arm extenders) is the crown base height. 

Like species code, this is an important measurement because it tells us things like how likely a fire is to move from the surface into the tree, or how much foliage is in the crown. However, also like species code, we can't learn this from the ALS data and it is a difficult thing to predict. But, let's start simple and train a random forest model to predict crown base height just from the height of the tree.

In [ ]:
# Split into training and test sets
# treelist["CBH"] = treelist["HT"] * (1 - treelist["CR"] / 100) #use this if CBH isn't included already
independent_variables = "HT"
dependent_variable = "CBH"
include_variables = independent_variables + dependent_variable
trees_train, trees_test = train_test_split(treelist[['HT', 'CBH']].dropna(), test_size=0.1)
print(f"Training set size: {len(trees_train)}")
print(f"Test set size: {len(trees_test)}")

# Train the model
model_cbh = RandomForestRegressor()
model_cbh.fit(trees_train[[independent_variables]], trees_train[dependent_variable])

# predict the model
predicted_cbh_test = model_cbh.predict(trees_test[[independent_variables]])
predicted_cbh = model_cbh.predict(ALS_treetops_filter[[independent_variables]])

# Evaluate model performance
print()
print(f"Model R^2: {model_cbh.score(trees_test[[independent_variables]], trees_test[dependent_variable]):.2f}")
print(f"Model RMSE: {((model_cbh.predict(trees_test[[independent_variables]]) - trees_test[dependent_variable])**2).mean()**0.5:.2f} feet")

# Compare the predicted CBH to the actual CBH.
fig, ax = plt.subplots()
trees_test["predicted_cbh"] = predicted_cbh_test
trees_test.plot.scatter(x="CBH", y="predicted_cbh", ax=ax)
upper_cbh_limit = max(trees_test["CBH"].max(), trees_test["predicted_cbh"].max()) + 1
ax.plot([0, upper_cbh_limit], [0, upper_cbh_limit], color='red')
ax.set_xlabel("Actual CBH (m)")
ax.set_ylabel("Predicted CBH (m)")
plt.show()

Note that live crown base height is often impacted by things like light availability, neighboring trees, and other landscape characteristics. 

In [ ]:
# save the predictions for tree metrics as a new treelist
predicted_treelist = pd.DataFrame({'treeID': ALS_treetops_filter.treeID,
                                   'HT': ALS_treetops_filter.HT,
                                   'DIA': predicted_dbh,
                                   'SPCD': predicted_spcd,
                                   'CBH': predicted_cbh,
                                   'X_4326': ALS_treetops_filter.X_4326,
                                   'Y_4326': ALS_treetops_filter.Y_4326,
                                  })

# predicted_treelist.to_csv('predicted_treelist.csv')

In [ ]:
predicted_treelist.head()

## Compare your results between the predicted treelist and the field data

Compare your results.

In [ ]:
plots_df = pd.read_csv('./01_plot_identification.csv')

plots_df.head()

In [ ]:
plots_intermediate = []
for srs in np.unique(plots_df.plot_coord_srs):
    plots_subset = plots_df[plots_df.plot_coord_srs == srs]
    plots_subset_gdf = gpd.GeoDataFrame(plots_subset, 
                                   geometry = gpd.points_from_xy(plots_subset.plot_coord_x, plots_subset.plot_coord_y), 
                                   crs = srs)

    #reproject to EPSG 5070
    plots_subset_gdf = plots_subset_gdf.to_crs(5070)
    
    plots_intermediate.append(plots_subset_gdf)

plots_gdf = pd.concat(plots_intermediate)
plots_gdf = plots_gdf.dropna(subset = 'plot_blk')

In [ ]:
#buffer the plots to get 1/10 acre plots
import numpy as np

plot_size = 1/10 #acre
acre_to_m2 = 4046.86
plot_size_m2 = plot_size * acre_to_m2
plot_radius = np.sqrt(plot_size_m2/np.pi)

plots_gdf = plots_gdf.set_geometry(plots_gdf.buffer(plot_radius))

In [ ]:
# filter the plot using the site name
plots_filtered = plots_gdf[plots_gdf.site_name == site_name]

In [ ]:
plots_filtered.plot()
plt.title(f'Plots at {site_name}')
plt.xlabel('X (meters)')
plt.ylabel('Y (meters)')

Now we will use spatial join to get the predicted trees for each plot


In [ ]:
# first we need to convert the predicted treelist from a dataframe to a geodataframe

predicted_treelist_gdf = gpd.GeoDataFrame(predicted_treelist, 
                                          geometry = gpd.points_from_xy(predicted_treelist.X_4326, predicted_treelist.Y_4326),
                                          crs = 4326)

# now convert predicted_treelist_gdf to the same crs as the plots

predicted_treelist_gdf = predicted_treelist_gdf.to_crs(plots_filtered.crs)

# Spatial join

predicted_treelist_plots = predicted_treelist_gdf.sjoin(plots_filtered)

predicted_treelist_plots.head()                                                                       

In [ ]:
# the plot names

plot_names = np.unique(predicted_treelist_plots.inventory_id)
print(plot_names)

Let's compare the treelists to the field data

In [ ]:
field_data_trees = pd.read_csv('./03_tree.csv')

In [ ]:
# Filter the trees by plot name
plot_id = 0
trees_filtered = field_data_trees[field_data_trees.inventory_id == plot_names[plot_id]]

trees_filtered.head()

In [ ]:
predicted_treelist_filtered = predicted_treelist_plots[predicted_treelist_plots.inventory_id == plot_names[plot_id]]
predicted_treelist_filtered.head()